In [1]:
import os
from PIL import Image
import imagehash
import shutil
import cv2

In [9]:
# Paramètres
INPUT_DIR = "../data/raw"
OUTPUT_DIR = '../data/processed'
CATEGORIES = ['acne', 'vitiligo', 'hyperpigmentation', 'nail_psoriasis', 'sjs_ten']
IMAGE_SIZE = (224, 224)
BLUR_THRESHOLD = 100

In [7]:
# Fonction pour détecter le flou
def is_blurry(image_path, threshold=BLUR_THRESHOLD):
    image = cv2.imread(image_path, 0)
    return cv2.Laplacian(image, cv2.CV_64F).var() < threshold

In [14]:
# Créer le dossier de sortie
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
processed_images = 0
raw_images = 0

In [ ]:
# Traitement par catégorie
for category in CATEGORIES:
    seen_hashes = set()
    input_path = os.path.join(INPUT_DIR, category)
    output_path = os.path.join(OUTPUT_DIR, category)
    os.makedirs(output_path, exist_ok=True)

    for filename in os.listdir(input_path):
        filepath = os.path.join(input_path, filename)
        raw_images += 1
        try:
            # Supprimer flou
            if is_blurry(filepath):
                continue

            # Supprimer doublons
            img = Image.open(filepath).convert('RGB')
            hash_img = imagehash.average_hash(img)
            if hash_img in seen_hashes:
                continue
            seen_hashes.add(hash_img)

            # Redimensionnement
            img = img.resize(IMAGE_SIZE)

            # Sauvegarde
            new_filename = os.path.join(output_path, filename)
            img.save(new_filename)
            processed_images += 1
            
            # Afficher le progrès
            print(f"Traitement de {filename} : {processed_images}/{raw_images} images traitées", end='\r')

        except Exception as e:
            print(f"Erreur avec {filepath}: {e}")
